In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets

In [ ]:
train = datasets.MNIST("", train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
test = datasets.MNIST("", train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 5385313.22it/s]


Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 158282.96it/s]


Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1289241.80it/s]


Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 9050132.43it/s]

Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw



In [ ]:
BATCH_SIZE = 32
train_dataset = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
test_dataset = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
class MNIST_NN(nn.Module):
  def __init__(self):
      super().__init__()
      self.input_layer = nn.Linear(28*28, 64)
      self.hidden_layer = nn.Linear(64, 64)
      self.output_layer = nn.Linear(64, 10)

  def forward(self, x):
    x = F.relu(self.input_layer(x))
    x = F.relu(self.hidden_layer(x))
    x = self.output_layer(x)
    return F.log_softmax(x, dim=1)

In [ ]:
model = MNIST_NN()

optimizer = optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 5
NUM_BATCHES = len(train_dataset)

for epoch in range(EPOCHS):
  for i, batch in enumerate(train_dataset):
    x, y = batch
    model.zero_grad()
    output = model(x.view(-1, 784))
    loss = F.nll_loss(output, y)
    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
      print (f"Epoch [{epoch+1}/{EPOCHS}], Step [{i+1}/{NUM_BATCHES}], Loss: {loss.item()}")

Epoch [1/5], Step [100/1875], Loss: 0.7480584383010864
Epoch [1/5], Step [200/1875], Loss: 0.3035137951374054
Epoch [1/5], Step [300/1875], Loss: 0.3697083294391632
Epoch [1/5], Step [400/1875], Loss: 0.16306354105472565
Epoch [1/5], Step [500/1875], Loss: 0.20854932069778442
Epoch [1/5], Step [600/1875], Loss: 0.22569096088409424
Epoch [1/5], Step [700/1875], Loss: 0.31162458658218384
Epoch [1/5], Step [800/1875], Loss: 0.16837386786937714
Epoch [1/5], Step [900/1875], Loss: 0.45358210802078247
Epoch [1/5], Step [1000/1875], Loss: 0.483830064535141
Epoch [1/5], Step [1100/1875], Loss: 0.08739211410284042
Epoch [1/5], Step [1200/1875], Loss: 0.20223617553710938
Epoch [1/5], Step [1300/1875], Loss: 0.09270298480987549
Epoch [1/5], Step [1400/1875], Loss: 0.1623680144548416
Epoch [1/5], Step [1500/1875], Loss: 0.031897272914648056
Epoch [1/5], Step [1600/1875], Loss: 0.13131262362003326
Epoch [1/5], Step [1700/1875], Loss: 0.14072059094905853
Epoch [1/5], Step [1800/1875], Loss: 0.033357

In [ ]:
def calculate_accuracy(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for batch in dataloader:
            x, y = batch

            # Check the size of the input and make sure it matches what we expect
            if x.shape[1:] != (1, 28, 28):
                raise RuntimeError(f"Expected input of size [batch_size, 1, 28, 28], but got {x.shape}")

            # Flatten the input only if it's in the correct shape
            x = x.view(-1, 784)

            output = model(x)

            # Get the predicted class by finding the index of the max log-probability
            _, predicted = torch.max(output, 1)

            # Count correct predictions
            correct += (predicted == y).sum().item()
            total += y.size(0)

    accuracy = correct / total * 100  # Convert to percentage
    return accuracy


In [ ]:
calculate_accuracy(model, test_dataset)
torch.save(model.state_dict(),"model.pt")